In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import visdom

import mxnet as mx
from mxnet import gluon
from mxnet import autograd
from mxnet import image

import sys
sys.path.append('../../resuneta/src')
sys.path.append('../../resuneta/nn/loss')
sys.path.append('../../resuneta/models')
sys.path.append('../../')
sys.path.append('../MXNet-ResUNeta/')

from bound_dist import get_distance, get_boundary
from loss import Tanimoto_with_dual_masked
from resunet_d6_causal_mtskcolor_ddist import *
from resunet_d7_causal_mtskcolor_ddist import *
from datasets import *

from sklearn.metrics import matthews_corrcoef

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def dice_coef(x, y):
    if type(x).__module__ == 'numpy':
        intersection = np.logical_and(x, y)
        return 2. * np.sum(intersection) / (np.sum(x) + np.sum(y))
    else:
        intersection = mx.ndarray.op.broadcast_logical_and(x, y)
        return 2. * mx.nd.sum(intersection) / (mx.nd.sum(x) + mx.nd.sum(y))

In [3]:
def visdom_visualize_batch(vis, img, extent, boundary, distance,
                           extent_pred, boundary_pred, distance_pred,
                           hsv, hsv_pred, mask, title="Train images"):

    img, extent, boundary, distance = img.asnumpy(), extent.asnumpy(), boundary.asnumpy(), distance.asnumpy()
    extent_pred, boundary_pred = extent_pred.asnumpy(), boundary_pred.asnumpy()
    distance_pred, hsv, hsv_pred = distance_pred.asnumpy(), hsv.asnumpy(), hsv_pred.asnumpy()
    mask = mask.asnumpy()

    # put everything in one window
    batch_size, nchannels, nrows, ncols = img.shape
    padding = 10
    items = [img, hsv, hsv_pred, extent, extent_pred, 
             boundary, boundary_pred, distance, distance_pred,
             mask]
    result = np.zeros((3, len(items)*nrows + (len(items)-1)*padding, batch_size*ncols + (batch_size-1)*padding))

    for j, item in enumerate(items):

        if item.shape[1] == 1:
            item = np.tile(item, (1,3,1,1)) * 255.

        if j == 1 or j == 2: # convert HSV to RGB
            item = np.moveaxis(item, 1, -1) * 255.
            for i in range(batch_size):
                item[i] = cv2.cvtColor(item[i].astype(np.uint8), cv2.COLOR_HSV2RGB)
            item = np.moveaxis(item, -1, 1)
            
        for i in range(batch_size):
            result[:, j*(nrows+padding):(j+1)*nrows+j*padding, i*(ncols+padding):(i+1)*ncols+i*padding] = item[i]
    vis.images(result, nrow=1, win=title, opts={'title': title})


In [4]:
def train_model(train_dataloader, model, tanimoto_dual, trainer, epoch, args):
    
    # initialize metrics
    cumulative_loss = 0
    accuracy = mx.metric.Accuracy()
    f1 = mx.metric.F1()
    mcc = mx.metric.MCC()
    dice = mx.metric.CustomMetric(feval=dice_coef, name="Dice")
    if args['ctx_name'] == 'cpu':
        ctx = mx.cpu()
    else:
        ctx = mx.gpu(args['gpu'])
    
    # training set
    for batch_i, (img, extent, boundary, distance, hsv, mask) in enumerate(
        tqdm(train_dataloader, desc='Training epoch {}'.format(epoch))):
        
        with autograd.record():

            img = img.as_in_context(ctx)
            extent = extent.as_in_context(ctx)
            boundary = boundary.as_in_context(ctx)
            distance = distance.as_in_context(ctx)
            hsv = hsv.as_in_context(ctx)
            mask = mask.as_in_context(ctx)
            nonmask = mx.nd.ones(extent.shape).as_in_context(ctx)
            
            logits, bound, dist, convc = model(img)
            
            # multi-task loss
            # TODO: wrap this in a custom loss function / class
            loss_extent = mx.nd.sum(1 - tanimoto_dual(logits, extent, mask))
            loss_boundary = mx.nd.sum(1 - tanimoto_dual(bound, boundary, mask))
            loss_distance = mx.nd.sum(1 - tanimoto_dual(dist, distance, mask))
            loss_hsv = mx.nd.sum(1 - tanimoto_dual(convc, hsv, nonmask)) # don't mask hsv

            loss = 0.25 * (loss_extent + loss_boundary + loss_distance + loss_hsv)
            
        loss.backward()
        trainer.step(args['batch_size'])
        cumulative_loss += mx.nd.sum(loss).asscalar()
        
        # update metrics based on every batch
#         print("logits.shape", logits.shape)
#         print("extent.shape", extent.shape)
#         print("mask.shape", mask.shape)
            
        # mask out unlabeled pixels            
#         print("made it here -3")
        logits_reshaped = logits.reshape((logits.shape[0], -1))
        extent_reshaped = extent.reshape((extent.shape[0], -1))
        mask_reshaped = mask.reshape((mask.shape[0], -1))
#         print("logits_reshaped.shape", logits_reshaped.shape)
        
        nonmask_idx = mx.np.nonzero(mask_reshaped.as_np_ndarray())
        nonmask_idx = mx.np.stack(nonmask_idx).as_nd_ndarray().as_in_context(ctx)
#         print("nonmask_idx", nonmask_idx)
        logits_masked = mx.nd.gather_nd(logits_reshaped, nonmask_idx)
#         print("logits_masked", logits_masked)
#         print("logits_masked.shape", logits_masked.shape)
        extent_masked = mx.nd.gather_nd(extent_reshaped, nonmask_idx)
#         print("extent_masked", extent_masked)
#         print("extent_masked.shape", extent_masked.shape)

#         print("made it here -1")
        # accuracy
        extent_predicted_classes = mx.nd.ceil(logits_masked - 0.5) # logits_masked[[0],:,:]
#         print("extent_predicted_classes", extent_predicted_classes)
#         print("extent_predicted_classes.shape", extent_predicted_classes.shape)
        
        accuracy.update(extent_masked, extent_predicted_classes)
#         print("made it here 1")
        
        # f1 score
#         prediction = logits[:,0,:,:].reshape(-1)
        probabilities = mx.nd.stack(1 - logits_masked, logits_masked, axis=1)
        f1.update(extent_masked, probabilities)
#         print("made it here 2")
        
        # MCC metric
        mcc.update(extent_masked, probabilities)
#         print("made it here 3")
        
        # Dice score
        dice.update(extent_masked, extent_predicted_classes)
#         print("made it here 4")
        
        # TODO: eccentricity
        # TODO: ...
        
        if batch_i % args['visdom_every'] == 0:
            visdom_visualize_batch(args['visdom'], img, extent, boundary, distance,
                                   logits, bound, dist, hsv, convc, mask)

    return cumulative_loss, accuracy, f1, mcc, dice

In [5]:
def evaluate_model(val_dataloader, model, tanimoto_dual, epoch, args):
    
    # initialize metrics
    cumulative_loss = 0
    accuracy = mx.metric.Accuracy()
    f1 = mx.metric.F1()
    mcc = mx.metric.MCC()
    dice = mx.metric.CustomMetric(feval=dice_coef, name="Dice")
    if args['ctx_name'] == 'cpu':
        ctx = mx.cpu()
    else:
        ctx = mx.gpu(args['gpu'])
    
    # validation set
    for batch_i, (img, extent, boundary, distance, hsv, mask) in enumerate(
        tqdm(val_dataloader, desc='Validation epoch {}'.format(epoch))):

        img = img.as_in_context(ctx)
        extent = extent.as_in_context(ctx)
        boundary = boundary.as_in_context(ctx)
        distance = distance.as_in_context(ctx)
        hsv = hsv.as_in_context(ctx)
        mask = mask.as_in_context(ctx)
        nonmask = mx.nd.ones(extent.shape).as_in_context(ctx)

        logits, bound, dist, convc = model(img)
        
        # multi-task loss
        # TODO: wrap this in a custom loss function / class
        loss_extent = mx.nd.sum(1 - tanimoto_dual(logits, extent, mask))
        loss_boundary = mx.nd.sum(1 - tanimoto_dual(bound, boundary, mask))
        loss_distance = mx.nd.sum(1 - tanimoto_dual(dist, distance, mask))
        loss_hsv = mx.nd.sum(1 - tanimoto_dual(convc, hsv, nonmask))

        loss = 0.25 * (loss_extent + loss_boundary + loss_distance + loss_hsv)
        
        # update metrics based on every batch
        cumulative_loss += mx.nd.sum(loss).asscalar()
        
        # update metrics based on every batch
        # mask out unlabeled pixels            
        logits_reshaped = logits.reshape((logits.shape[0], -1))
        extent_reshaped = extent.reshape((extent.shape[0], -1))
        mask_reshaped = mask.reshape((mask.shape[0], -1))
        
        nonmask_idx = mx.np.nonzero(mask_reshaped.as_np_ndarray())
        nonmask_idx = mx.np.stack(nonmask_idx).as_nd_ndarray().as_in_context(ctx)
        logits_masked = mx.nd.gather_nd(logits_reshaped, nonmask_idx)
        extent_masked = mx.nd.gather_nd(extent_reshaped, nonmask_idx)

        # accuracy
        extent_predicted_classes = mx.nd.ceil(logits_masked - 0.5)
        accuracy.update(extent_masked, extent_predicted_classes)
        
        # f1 score
        probabilities = mx.nd.stack(1 - logits_masked, logits_masked, axis=1)
        f1.update(extent_masked, probabilities)
        
        # MCC metric
        mcc.update(extent_masked, probabilities)
        
        # Dice score
        dice.update(extent_masked, extent_predicted_classes)
        
        # TODO: eccentricity
        # TODO: ...
        
        if batch_i % args['visdom_every'] == 0:
            visdom_visualize_batch(args['visdom'], img, extent, boundary, distance,
                                   logits, bound, dist, hsv, convc, mask, title="Val images")
        
    return cumulative_loss, accuracy, f1, mcc, dice

# Africa datasets

In [6]:
def run_africa(country, train_names, val_names, test_names, 
               train_names_label, val_names_label, test_names_label,
               trained_model=None,
               epochs=100, lr=0.001, lr_decay=None, n_filters=16, batch_size=8,
               n_classes=1, model_type='resunet-d6', month='janFebMar',
               codes_to_keep=[1, 2],
               folder_suffix='',
               boundary_kernel_size=3,
               ctx_name='cpu',
               gpu_id=0):
    
    # Set MXNet ctx
    if ctx_name == 'cpu':
        ctx = mx.cpu()
    elif ctx_name == 'gpu':
        ctx = mx.gpu(gpu_id)
    
    # Set up names of directories and paths for saving
    if trained_model is None:
        folder_name = model_type+'_'+month+'_nfilter-'+str(n_filters)+ \
                      '_bs-'+str(batch_size)+'_lr-'+str(lr)+folder_suffix
        if lr_decay:
            folder_name = folder_name + '_lrdecay-'+str(lr_decay)
            
        # define model
        if model_type == 'resunet-d6':
            model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes)
        elif model_type == 'resunet-d7':
            model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes)
        model.initialize()
        model.hybridize()
        model.collect_params().reset_ctx(ctx)
        
    else:
        folder_name = model_type+'_'+month+'_nfilter-'+str(n_filters)+ \
                      '_bs-'+str(batch_size)+'_lr-'+str(lr)+folder_suffix+'_finetuned'
        if model_type == 'resunet-d6':
            model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes)
        elif model_type == 'resunet-d7':
            model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes)
        model.load_parameters(trained_model, ctx=ctx)
        
    save_path = os.path.join('../experiments/', country, folder_name)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_model_name = os.path.join(save_path, "model.params")
    
    # Visdom
    env_name = country + '_' + folder_name
    vis = visdom.Visdom(port=8097, env=env_name)
    
    # Arguments
    args = {}
    args['batch_size'] = batch_size
    args['ctx_name'] = ctx_name
    args['gpu'] = gpu_id
    args['visdom'] = vis
    args['visdom_every'] = 20

    # Define train/val/test splits
    train_dataset = PlanetDatasetWithClassesFullPathsMasked(
        fold='train', 
        image_names=train_names, 
        label_names=train_names_label, 
        classes=codes_to_keep,
        boundary_kernel_size=boundary_kernel_size)
    val_dataset = PlanetDatasetWithClassesFullPathsMasked(
        fold='val', 
        image_names=val_names, 
        label_names=val_names_label, 
        classes=codes_to_keep,
        boundary_kernel_size=boundary_kernel_size)
    test_dataset = PlanetDatasetWithClassesFullPathsMasked(
        fold='test', 
        image_names=test_names, 
        label_names=test_names_label, 
        classes=codes_to_keep,
        boundary_kernel_size=boundary_kernel_size)

    train_dataloader = gluon.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = gluon.data.DataLoader(test_dataset, batch_size=batch_size)

    # define loss function
    tanimoto_dual = Tanimoto_with_dual_masked()
    if lr_decay:
        schedule = mx.lr_scheduler.FactorScheduler(step=1, factor=lr_decay)
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr, lr_scheduler=schedule)
    else:
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr)
    trainer = gluon.Trainer(model.collect_params(), optimizer=adam_optimizer)

    # containers for metrics to log
    train_metrics = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                     'train_mcc': [], 'train_dice': []}
    val_metrics = {'val_loss': [], 'val_acc': [], 'val_f1': [], 
                   'val_mcc': [], 'val_dice': []}
    best_mcc = 0.0

    # training loop
    for epoch in range(1, epochs+1):

        # training set
        train_loss, train_accuracy, train_f1, train_mcc, train_dice = train_model(
            train_dataloader, model, tanimoto_dual, trainer, epoch, args)

        # training set metrics
        train_loss_avg = train_loss / len(train_dataset)
        train_metrics['train_loss'].append(train_loss_avg)
        train_metrics['train_acc'].append(train_accuracy.get()[1])
        train_metrics['train_f1'].append(train_f1.get()[1])
        train_metrics['train_mcc'].append(train_mcc.get()[1])
        train_metrics['train_dice'].append(train_dice.get()[1])

        # validation set
        val_loss, val_accuracy, val_f1, val_mcc, val_dice = evaluate_model(
            val_dataloader, model, tanimoto_dual, epoch, args)

        # validation set metrics
        val_loss_avg = val_loss / len(val_dataset)
        val_metrics['val_loss'].append(val_loss_avg)
        val_metrics['val_acc'].append(val_accuracy.get()[1])
        val_metrics['val_f1'].append(val_f1.get()[1])
        val_metrics['val_mcc'].append(val_mcc.get()[1])
        val_metrics['val_dice'].append(val_dice.get()[1])

        print("Epoch {}:".format(epoch))
        print("    Train loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            train_loss_avg, train_accuracy.get()[1], train_f1.get()[1], train_mcc.get()[1], train_dice.get()[1]))
        print("    Val loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            val_loss_avg, val_accuracy.get()[1], val_f1.get()[1], val_mcc.get()[1], val_dice.get()[1]))

        # save model based on best MCC metric
        if val_mcc.get()[1] > best_mcc:
            model.save_parameters(save_model_name)
            best_mcc = val_mcc.get()[1]

        # save metrics
        metrics = pd.concat([pd.DataFrame(train_metrics), pd.DataFrame(val_metrics)], axis=1)
        metrics.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)

        # visdom
        vis.line(Y=np.stack([train_metrics['train_loss'], val_metrics['val_loss']], axis=1), 
                 X=np.arange(1, epoch+1), win="Loss", 
                 opts=dict(legend=['train loss', 'val loss'], markers=False, title="Losses",
                           xlabel="Epoch", ylabel="Loss")
                )
        vis.line(Y=np.stack([train_metrics['train_mcc'], val_metrics['val_mcc']], axis=1), 
                 X=np.arange(1, epoch+1), win="MCC", 
                 opts=dict(legend=['train MCC', 'val MCC'], markers=False, title="MCC",
                           xlabel="Epoch", ylabel="MCC")
                )


In [7]:
# # ============================ #
# # user-specified hyperparameters
# # ============================ #
# country = 'partial-france'
# epochs = 100
# lr = 0.001
# lr_decay = None
# n_filters = 16
# batch_size = 8
# n_classes = 1
# model_type = 'resunet-d6'
# month = 'aprJulOctSeparate'
# codes_to_keep = [1]
# ctx_name = 'gpu'
# gpu_id = 0
# boundary_kernel_size = (2,2)

# # trained_model = '../experiments/france/sherrie10k/' + \
# #     'resunet-d6_2019_10_class-notreeexceptvines_nfilter-16_bs-8_lr-0.001_1x-8x-downsampled/model.params'
# trained_model = None
# splits_path = '../data/splits/sherrie10k_planetImagery_splits_20x20_4x-downsampled.csv'
# splits_df = pd.read_csv(splits_path, dtype=str)

# # get all img and labels
# all_img_names = []
# all_label_names = []
# img_dir = '../data/planet/france/sherrie10k/monthly_mosaics_renamed_clipped_merged/1250px/4x_downsample/'
# label_dir = '../data/planet/france/sherrie10k/extent_labels/1250px_1field/4x_downsample/'

# label_folder_imgs = sorted(os.listdir(label_dir))
# for month in ['2019_04', '2019_07', '2019_10']:
#     for label_name in label_folder_imgs:
#         img_name = label_name.split('.')[0] + '_' + month + '.tif'
#         img_path = os.path.join(img_dir, month, img_name)
#         all_img_names.append(img_path)
#         label_path = os.path.join(label_dir, label_name)
#         all_label_names.append(label_path)

# # split imgs and labels into train/val/test
# all_images = pd.DataFrame({'img_path': all_img_names})
# all_images['image_id'] = all_images['img_path'].str.split('/').apply(
#     lambda x: x[-1]).str.split('.').apply(
#     lambda x: x[0]).str.split('_').apply(
#     lambda x: x[0])
# all_images = all_images.merge(splits_df[['image_id', 'fold']], on='image_id', how='left')
# train_names = all_images[all_images['fold'] == 'train']['img_path'].values
# val_names = all_images[all_images['fold'] == 'val']['img_path'].values
# test_names = all_images[all_images['fold'] == 'test']['img_path'].values

# all_labels = pd.DataFrame({'label_path': all_label_names})
# all_labels['image_id'] = all_labels['label_path'].str.split('/').apply(
#     lambda x: x[-1]).str.split('.').apply(
#     lambda x: x[0])
# all_labels = all_labels.merge(splits_df[['image_id', 'fold']], on='image_id', how='left')
# train_names_label = all_labels[all_labels['fold'] == 'train']['label_path'].values
# val_names_label = all_labels[all_labels['fold'] == 'val']['label_path'].values
# test_names_label = all_labels[all_labels['fold'] == 'test']['label_path'].values

# # ============================ #

# run_africa(country, train_names, val_names, test_names,
#            train_names_label, val_names_label, test_names_label,
#            trained_model=trained_model,
#            epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
#            n_classes=n_classes, model_type=model_type, month=month,
#            codes_to_keep=codes_to_keep, 
#            ctx_name=ctx_name,
#            gpu_id=gpu_id, 
#            folder_suffix='_4x-downsampled',
#            boundary_kernel_size=boundary_kernel_size)

In [8]:
# ============================ #
# user-specified hyperparameters
# ============================ #
country = 'partial-france'
epochs = 100
lr = 0.001
lr_decay = None
n_filters = 16
batch_size = 8
n_classes = 1
model_type = 'resunet-d6'
month = 'aprJulOctSeparate'
# codes_to_keep = [1]
codes_to_keep = list(range(1,10)) + [11,14,15,16,17,18,19,21,24,25,26,28]
ctx_name = 'gpu'
gpu_id = 2
boundary_kernel_size = (2,2)

# trained_model = '../experiments/france/sherrie10k/' + \
#     'resunet-d6_2019_10_class-notreeexceptvines_nfilter-16_bs-8_lr-0.001_1x-8x-downsampled/model.params'
trained_model = None
splits_path = '../data/splits/sherrie10k_planetImagery_splits_20x20_4x-downsampled.csv'
splits_df = pd.read_csv(splits_path)
splits_df['image_id'] = splits_df['image_id'].astype(str).str.zfill(5)

# get all img and labels
all_img_names = []
all_label_names = []
img_dir = '../data/planet/france/sherrie10k/monthly_mosaics_renamed_clipped_merged/1250px/4x_downsample/'
label_dir = '../data/planet/france/sherrie10k/extent_labels/1250px/4x_downsample/'

label_folder_imgs = sorted(os.listdir(label_dir))
for month in ['2019_04', '2019_07', '2019_10']:
    for label_name in label_folder_imgs:
        img_name = label_name.split('.')[0] + '_' + month + '.tif'
        img_path = os.path.join(img_dir, month, img_name)
        all_img_names.append(img_path)
        label_path = os.path.join(label_dir, label_name)
        all_label_names.append(label_path)

# split imgs and labels into train/val/test
all_images = pd.DataFrame({'img_path': all_img_names})
all_images['image_id'] = all_images['img_path'].str.split('/').apply(
    lambda x: x[-1]).str.split('.').apply(
    lambda x: x[0]).str.split('_').apply(
    lambda x: x[0])
all_images = all_images.merge(splits_df[['image_id', 'fold']], on='image_id', how='left')
train_names = all_images[all_images['fold'] == 'train']['img_path'].values
val_names = all_images[all_images['fold'] == 'val']['img_path'].values
test_names = all_images[all_images['fold'] == 'test']['img_path'].values

all_labels = pd.DataFrame({'label_path': all_label_names})
all_labels['image_id'] = all_labels['label_path'].str.split('/').apply(
    lambda x: x[-1]).str.split('.').apply(
    lambda x: x[0])
all_labels = all_labels.merge(splits_df[['image_id', 'fold']], on='image_id', how='left')
train_names_label = all_labels[all_labels['fold'] == 'train']['label_path'].values
val_names_label = all_labels[all_labels['fold'] == 'val']['label_path'].values
test_names_label = all_labels[all_labels['fold'] == 'test']['label_path'].values

# ============================ #

run_africa(country, train_names, val_names, test_names,
           train_names_label, val_names_label, test_names_label,
           trained_model=trained_model,
           epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
           n_classes=n_classes, model_type=model_type, month=month,
           codes_to_keep=codes_to_keep, 
           ctx_name=ctx_name,
           gpu_id=gpu_id, 
           folder_suffix='_4x-downsampled_allfields_n6759',
           boundary_kernel_size=boundary_kernel_size)

depth:= 0, nfilters: 16
depth:= 1, nfilters: 32
depth:= 2, nfilters: 64
depth:= 3, nfilters: 128
depth:= 4, nfilters: 256
depth:= 5, nfilters: 512
depth:= 6, nfilters: 256
depth:= 7, nfilters: 128
depth:= 8, nfilters: 64
depth:= 9, nfilters: 32
depth:= 10, nfilters: 16


Training epoch 14:   0%|          | 0/2535 [00:00<?, ?it/s]

Epoch 13:
    Train loss 0.162, accuracy 0.834, F1-score 0.889, MCC: 0.547, Dice: 0.889
    Val loss 0.157, accuracy 0.845, F1-score 0.899, MCC: 0.558, Dice: 0.899


Validation epoch 14:  37%|███▋      | 215/580 [01:01<01:28,  4.11it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Training epoch 17:   0%|          | 0/2535 [00:00<?, ?it/s]

Epoch 16:
    Train loss 0.161, accuracy 0.836, F1-score 0.890, MCC: 0.550, Dice: 0.890
    Val loss 0.155, accuracy 0.840, F1-score 0.894, MCC: 0.552, Dice: 0.894


...

Training epoch 36:   0%|          | 0/2535 [00:00<?, ?it/s]

Epoch 35:
    Train loss 0.159, accuracy 0.839, F1-score 0.893, MCC: 0.558, Dice: 0.893
    Val loss 0.149, accuracy 0.849, F1-score 0.900, MCC: 0.578, Dice: 0.900


Validation epoch 36: 100%|██████████| 580/580 [04:17<00:00,  2.25it/s]

Epoch 36:
    Train loss 0.159, accuracy 0.839, F1-score 0.892, MCC: 0.558, Dice: 0.892
    Val loss 0.147, accuracy 0.844, F1-score 0.894, MCC: 0.585, Dice: 0.894


FileNotFoundError: [Errno 2] No such file or directory: '../experiments/partial-france/resunet-d6_2019_10_nfilter-16_bs-8_lr-0.001_4x-downsampled_allfields_n6759/metrics.csv'

# Appendix: dev

In [68]:
ctx = mx.cpu()
mask_j = mx.nd.array([[[1,0,0],
                       [0,1,0]],
                      [[0,0,1],
                       [0,0,0]]])
mask_j = mask_j.reshape((mask_j.shape[0], -1))

In [70]:
mask_j


[[1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]]
<NDArray 2x6 @cpu(0)>

In [84]:
nonmask_idx = mx.np.nonzero(mask_j.as_np_ndarray())
nonmask_idx = mx.np.stack(nonmask_idx).as_nd_ndarray().as_in_context(ctx)

In [85]:
mx.nd.gather_nd(mask_j, nonmask_idx)


[1. 1. 1.]
<NDArray 3 @cpu(0)>

In [4]:
tanimoto = Tanimoto_with_dual_masked()
test_pred = mx.nd.array([[[[0.1,0.1,0.8,0.9]]]])
test_label = mx.nd.array([[[[0,0,1,0]]]])
test_mask = mx.nd.array([[[[0,1,1,1]]]])
tanimoto(test_pred, test_label, test_mask)


[0.5097876]
<NDArray 1 @cpu(0)>

In [18]:
for row in mx.nd.array([[0,1,0],
             [0,0,0]]):
    print(row)
    
mx.nd.array([[0,1,0],
             [0,0,0]]).reshape((-1,))


[0. 1. 0.]
<NDArray 3 @cpu(0)>

[0. 0. 0.]
<NDArray 3 @cpu(0)>



[0. 1. 0. 0. 0. 0.]
<NDArray 6 @cpu(0)>

In [29]:
mx.npx.set_np()
mx.np.nonzero(mx.nd.array([[0,1,0],
                           [0,0,0]]).as_np_ndarray())[0]

mxnet.numpy.ndarray

In [48]:
test_array = mx.nd.array([[ 0.1, 0.9, 0.5],
                          [ 0.2, 0.3, 0.7]])
test_array2 = mx.nd.array([[0, 2, 0],
                           [0, 1, 0]])
nonmask_idx = mx.np.nonzero(test_array2.as_np_ndarray())
test_result = mx.nd.gather_nd(test_array, mx.nd.array(nonmask_idx))
test_result2 = mx.nd.gather_nd(test_array2, mx.nd.array(nonmask_idx))

In [49]:
test_result


[0.9 0.3]
<NDArray 2 @cpu(0)>

In [50]:
test_result2 == 2


[1. 0.]
<NDArray 2 @cpu(0)>